# Geometry of QAOA landscape

by Pantita Palittapongarnpim

### Package Versions

QuTiP: 4.5.2

Python: 3.8.6
    
Plotly: 

In [1]:
import numpy as np
import qutip as qt
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## QAOA Classes

In [2]:
class Graph:
    def __init__(self, total_qubit, J):
        self.total_qubit=total_qubit #integer
        self.J=J #matrix: undirected (symmetric)
        
    def Ham(self):
        Ham_out=0;
        for i in range(self.total_qubit):
            for j in range(i,self.total_qubit):
                opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
                Ham_out=Ham_out+J[i][j]*opr
        return(Ham_out)

In [3]:
class QAOA:
    def __init__(self, gamma, beta, p, graph):
        ## what does graph has to contain:
        # total_qubit -- integer
        # J -- double matrix        
        self.gamma=gamma
        self.beta=beta
        self.p=p
        self.J=graph.J
        self.total_qubit=graph.total_qubit
        
    def Uz(self,i,j,l):
        opr=qt.tensor([qt.identity(2) if (k != i and k != j) else qt.sigmaz() for k in range(self.total_qubit)])
        Hvar=-1j*self.gamma[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def Ux(self,i,l):
        opr=qt.tensor([qt.identity(2) if (k != i) else qt.sigmax() for k in range(self.total_qubit)])
        Hvar=-1j*self.beta[l-1]*J[i][j]*opr
        return(Hvar.expm())
    
    def SumX(self):
        opr=sum(qt.tensor([qt.identity(2) if (k != i) else qt.sigmax() for k in range(self.total_qubit)]) for i in range(self.total_qubit))
        return(opr)
    
    def circuit(self):
        Had=qt.tensor([qt.qip.operations.snot() for k in range(self.total_qubit)])
        qaoa_circ=Had
        for l in range(self.p):
            #operate the Uz gates
            for i in range(self.total_qubit):
                for j in range(i,self.total_qubit):
                    if i!= j:
                        qaoa_circ=self.Uz(i,j,l)*qaoa_circ
            #operate the Ux gates
            for i in range(self.total_qubit):
                qaoa_circ=self.Ux(i,l)*qaoa_circ
        return(qaoa_circ)
    
    def state(self):
        gnd=qt.basis(2,0)
        state=qt.tensor([gnd for i in range(self.total_qubit)])
        state=self.circuit()*state
        return(state)
 

## Objective functions

Now that we have the simulation of the QAOA circuit, we need to optimize the parameters, namely gamma and beta array.

But before that, we need to pick the objective function we are going to use.

__Average Energy__

This is the traditional objective function for QAOA

__Gibbs__

This one is proposed by Li et al.

__Average Gibbs__

Actually doesn't have a name, proposed by P'Thip.

In [4]:
#so this is our fun in scipy.optimize.minimize
# the form has to be fun(x, *arg)
# x here has to be gamma and beta
# arg* has to be graph and p

def AveEnergy(x, *arg):
    p=arg[0]
    graph=arg[1]    
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]

    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    return((output.dag()*graph.Ham()*output).tr())

def Gibbs(x, *arg):
    p=arg[0]
    graph=arg[1]    
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]

    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    f=-np.log((output.dag()*(-eta*graph.Ham()).expm()*output).tr())
    return(f)

def AveGibbs(x, *arg):
    p=arg[0]
    graph=arg[1]    
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]

    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    f=-np.log((output.dag()*graph.Ham()*(-eta*graph.Ham()).expm()*output).tr())
    return(f)

## Setting up problem

In [5]:
total_qubit=3 #number of qubit
p=2 #number of layers

#defining the problem
J=np.random.rand(total_qubit,total_qubit) #creating the couping parameters,still directional in this instance)
for i in range(total_qubit):
    for j in range(i,total_qubit):
        temp=(J[i][j]+J[j][i])/2.0
        J[i][j]=temp
        J[j][i]=temp
#Now J is undirectional

In [14]:
graph = Graph(total_qubit,J)
arg=(p,graph)

#declaring QAOA parameters
gamma_in=np.random.rand(p)
beta_in=np.random.rand(p)
##How do we link this to the parameter space we want to visualize?

#Let's select the space here
#Randomly, as a test
#gamma_d=np.random.randint(0,len(gamma_in))
#beta_d=np.random.randint(0,len(beta_in))

#Define the space
gamma0=np.linspace(-2, 2, 15)
beta0=np.linspace(-2, 2, 15)
X, Y = np.meshgrid(gamma0, beta0)

z=np.zeros((len(gamma0),len(beta0)))

eta=0.1
    

Calculating landscape for all parameter in the range and visualize the result

In [15]:
fig = make_subplots(
    rows=p, cols=p,
#    specs=[[{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}], #Still need to automate this part
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}],
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}]])

     specs=[[{'type': 'surface'},{'type': 'surface'}], #Still need to automate this part
           [{'type': 'surface'},{'type': 'surface'}]])

for gamma_d in range(len(gamma_in)):
    for beta_d in range(len(gamma_in)):
        landscape=np.zeros((len(gamma0),len(beta0)))
        for k in range(len(gamma0)):
            for l in range(len(beta0)):
                #creating x0
                temp_gamma=gamma_in
                temp_gamma[gamma_d]=gamma0[k]
                temp_beta=beta_in
                temp_beta[beta_d]=beta0[l]
                x0=np.append(temp_gamma,temp_beta)
                #print(x0)

                #res=minimize(AveEnergy,x0,arg,method='Nelder-Mead')
                #z[k][l]=res.fun
                landscape[k][l]=AveEnergy(x0,*arg)
                #print(z[k][l])
        fig.add_trace(
        go.Surface(x=X, y=Y, z=landscape, showscale=False),
        row=gamma_d+1, col=beta_d+1)
                
fig.update_layout(
    title_text='Energy Landscape',
    height=1000,
    width=1000
)

fig.show()

### Calculating the derivatives

In [56]:
def loop_k(k,x):
    while k>=len(x):
        k=k-len(x)
    return(k)

def dstate(x,k,*arg):
    p=arg[0]
    graph=arg[1]    
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]

    qaoa = QAOA(gamma,beta,p,graph)
    
    k=loop_k(k,x)
    
    div_k=k%p
    
    Had=qt.tensor([qt.qip.operations.snot() for k in range(qaoa.total_qubit)])
    qaoa_circ=Had
    for l in range(qaoa.p):
        if div_k != l:
            #operate the Uz gates
            for i in range(qaoa.total_qubit):
                for j in range(i,qaoa.total_qubit):
                    if i!= j:
                        qaoa_circ=qaoa.Uz(i,j,l)*qaoa_circ
            #operate the Ux gates
            for i in range(qaoa.total_qubit):
                qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
        else:
            #operate the Uz gates
            for i in range(qaoa.total_qubit):
                for j in range(i,qaoa.total_qubit):
                    if i!= j:
                        qaoa_circ=qaoa.Uz(i,j,l)*qaoa_circ
            if k<=p: #this is a derivative on Uz
                qaoa_circ=-1j*graph.Ham()*qaoa_circ
                #operate the Ux gates
                for i in range(qaoa.total_qubit):
                    qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
            else: #this is a derivative on Ux
                for i in range(qaoa.total_qubit):
                    qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
                qaoa_circ=-1j*qaoa.SumX()*qaoa_circ
    
    gnd=qt.basis(2,0)
    state=qt.tensor([gnd for i in range(qaoa.total_qubit)])
    state=qaoa_circ*state
    
    return(state)

def d2state(x,k1,k2,*arg):
    p=arg[0]
    graph=arg[1]
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]
    
    qaoa = QAOA(gamma,beta,p,graph)
    
    k_n=[loop_k(k1,x),loop_k(k2,x)]
    div_k=[k1%p,k2%p]
    
    Had=qt.tensor([qt.qip.operations.snot() for k in range(qaoa.total_qubit)])
    qaoa_circ=Had    
    for l in range(qaoa.p):
        if l!= div_k[0] or l!=div_k[1]:
            #operate the Uz gates
            for i in range(qaoa.total_qubit):
                for j in range(i,qaoa.total_qubit):
                    if i!= j:
                        qaoa_circ=qaoa.Uz(i,j,l)*qaoa_circ
            #operate the Ux gates
            for i in range(qaoa.total_qubit):
                qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
        elif l==div_k[0] and div_k[0]==div_k[1]:
            if k_n[0]!=k_n[1]:
                for i in range(qaoa.total_qubit):
                    for j in range(i,qaoa.total_qubit):
                        if i!= j:
                            qaoa_circ=qaoa.Uz(i,j,l)*qaoa_circ
                #this is a derivative on Uz
                qaoa_circ=-1j*graph.Ham()*qaoa_circ
                    #operate the Ux gates
                for i in range(qaoa.total_qubit):
                    qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
                #this is a derivative on Ux
                qaoa_circ=-1j*qaoa.SumX()*qaoa_circ
            else:
                k=k_n[0]
                for i in range(qaoa.total_qubit):
                    for j in range(i,qaoa.total_qubit):
                        if i!= j:
                            qaoa_circ=qaoa.Uz(i,j,l)*qaoa_circ
                if k<=p: #this is a derivative on Uz
                    qaoa_circ=-1*graph.Ham()*graph.Ham()*qaoa_circ
                    #operate the Ux gates
                    for i in range(qaoa.total_qubit):
                        qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
                else: #this is a derivative on Ux
                    for i in range(qaoa.total_qubit):
                        qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
                    qaoa_circ=-1*qaoa.SumX()*qaoa.SumX()*qaoa_circ
        else:
            if l==div_k[0]:k=k_n[0]
            else: k=k_n[1]   
            #operate the Uz gates
            for i in range(qaoa.total_qubit):
                for j in range(i,qaoa.total_qubit):
                    if i!= j:
                        qaoa_circ=qaoa.Uz(i,j,l)*qaoa_circ
            if k<=p: #this is a derivative on Uz
                qaoa_circ=-1j*graph.Ham()*qaoa_circ
                #operate the Ux gates
                for i in range(qaoa.total_qubit):
                    qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
            else: #this is a derivative on Ux
                for i in range(qaoa.total_qubit):
                    qaoa_circ=qaoa.Ux(i,l)*qaoa_circ
                qaoa_circ=-1j*qaoa.SumX()*qaoa_circ
    
    gnd=qt.basis(2,0)
    state=qt.tensor([gnd for i in range(qaoa.total_qubit)])
    state=qaoa_circ*state
    
    return(state)

def dAveEnergy(x,k,*arg):
    p=arg[0]
    graph=arg[1]
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]
    
    dpsi= dstate(x,k,*arg)
    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    df=(dpsi.dag()*graph.Ham()*output-output.dag()*graph.Ham()*dpsi).tr()
    
    return(df)

def d2AveEnergy(x,k1,k2,*arg):
    p=arg[0]
    graph=arg[1]
    y=np.array_split(x,2)
    gamma=y[0]
    beta=y[1]
    
    dpsi1= dstate(x,k1,*arg)
    dpsi2= dstate(x,k2,*arg)
    d2psi=d2state(x,k1,k2,*arg)
    qaoa = QAOA(gamma,beta,p,graph)
    output=qaoa.state()
    
    d2f=d2psi.dag()*graph.Ham()*output-output.dag()*graph.Ham()*d2psi+dpsi1.dag()*graph.Ham()*dpsi2-dpsi2.dag()*graph.Ham()*dpsi1
    
    return(d2f.tr())

Testing the  derivative

In [53]:
gamma_in=np.random.rand(p)
beta_in=np.random.rand(p)

x0=np.append(gamma_in,beta_in)

print(d2state(x0,0,0,*arg))
print(dstate(x0,0,*arg))

#print(dAveEnergy(x0,k,*arg))


Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[ 0.9874105 +0.60720141j]
 [ 0.05719349-0.30171733j]
 [-0.35845196-0.17941717j]
 [ 0.26868096-0.11141859j]
 [ 1.23915524-0.4334061j ]
 [ 0.09367936-0.19420143j]
 [ 0.23547184-1.06632415j]
 [-0.17939537+0.18804037j]]
Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[ 0.32635759+0.30799276j]
 [ 0.08065738-0.01165422j]
 [ 0.47501431+0.36522779j]
 [-0.08019173+0.12239508j]
 [-0.06283065+0.62979965j]
 [ 0.38957689+0.06697115j]
 [ 0.24419065-0.0436997j ]
 [ 0.27732129+0.14128736j]]


In [61]:
fig = make_subplots(
    rows=p, cols=p,
#    specs=[[{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}], #Still need to automate this part
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}],
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}]])

     specs=[[{'type': 'surface'},{'type': 'surface'}], #Still need to automate this part
           [{'type': 'surface'},{'type': 'surface'}]])

a=0
for gamma_d in range(len(gamma_in)):
    for beta_d in range(len(gamma_in)):
        landscape=np.zeros((len(gamma0),len(beta0)))
        for k in range(len(gamma0)):
            for l in range(len(beta0)):
                #creating x0
                temp_gamma=gamma_in
                temp_gamma[gamma_d]=gamma0[k]
                temp_beta=beta_in
                temp_beta[beta_d]=beta0[l]
                x0=np.append(temp_gamma,temp_beta)
                #print(x0)

                #res=minimize(AveEnergy,x0,arg,method='Nelder-Mead')
                #z[k][l]=res.fun
                landscape[k][l]=np.absolute(dAveEnergy(x0,a,*arg))
                #print(z[k][l])
        fig.add_trace(
        go.Surface(x=X, y=Y, z=landscape, showscale=False),
        row=gamma_d+1, col=beta_d+1)
                
fig.update_layout(
    title_text='Energy Landscape',
    height=800,
    width=800
)

fig.show()

In [62]:
fig = make_subplots(
    rows=p, cols=p,
#    specs=[[{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}], #Still need to automate this part
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}],
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}]])

     specs=[[{'type': 'surface'},{'type': 'surface'}], #Still need to automate this part
           [{'type': 'surface'},{'type': 'surface'}]])

a=1
for gamma_d in range(len(gamma_in)):
    for beta_d in range(len(gamma_in)):
        landscape=np.zeros((len(gamma0),len(beta0)))
        for k in range(len(gamma0)):
            for l in range(len(beta0)):
                #creating x0
                temp_gamma=gamma_in
                temp_gamma[gamma_d]=gamma0[k]
                temp_beta=beta_in
                temp_beta[beta_d]=beta0[l]
                x0=np.append(temp_gamma,temp_beta)
                #print(x0)

                #res=minimize(AveEnergy,x0,arg,method='Nelder-Mead')
                #z[k][l]=res.fun
                landscape[k][l]=np.absolute(dAveEnergy(x0,a,*arg))
                #print(z[k][l])
        fig.add_trace(
        go.Surface(x=X, y=Y, z=landscape, showscale=False),
        row=gamma_d+1, col=beta_d+1)
                
fig.update_layout(
    title_text='Energy Landscape',
    height=800,
    width=800
)

fig.show()

In [63]:
fig = make_subplots(
    rows=p, cols=p,
#    specs=[[{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}], #Still need to automate this part
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}],
#           [{'type': 'surface'},{'type': 'surface'}, {'type': 'surface'}]])

     specs=[[{'type': 'surface'},{'type': 'surface'}], #Still need to automate this part
           [{'type': 'surface'},{'type': 'surface'}]])

a1=0
a2=0
for gamma_d in range(len(gamma_in)):
    for beta_d in range(len(gamma_in)):
        landscape=np.zeros((len(gamma0),len(beta0)))
        for k in range(len(gamma0)):
            for l in range(len(beta0)):
                #creating x0
                temp_gamma=gamma_in
                temp_gamma[gamma_d]=gamma0[k]
                temp_beta=beta_in
                temp_beta[beta_d]=beta0[l]
                x0=np.append(temp_gamma,temp_beta)
                #print(x0)

                #res=minimize(AveEnergy,x0,arg,method='Nelder-Mead')
                #z[k][l]=res.fun
                landscape[k][l]=np.absolute(d2AveEnergy(x0,a1,a2,*arg))
                #print(z[k][l])
        fig.add_trace(
        go.Surface(x=X, y=Y, z=landscape, showscale=False),
        row=gamma_d+1, col=beta_d+1)
                
fig.update_layout(
    title_text='Energy Landscape',
    height=800,
    width=800
)

fig.show()